In [1]:
# Data Analysis
import warnings 
warnings.filterwarnings('ignore')
    
import pandas as pd
import numpy as np
import os 
import missingno as msno
import matplotlib.pyplot as plt
    
# Data View
pd.options.display.max_columns = 200

# Import Basic Visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

os.chdir('C:\\Users\\jason\\OneDrive\\Desktop\\NCU\\8535_Univariate Analysis\\Week 4')

In [2]:
#Extract dataset
df_raw= pd.read_csv("C:\\Users\\jason\\OneDrive\\Desktop\\NCU\8535_Univariate Analysis\\Week 4\\Baltimore_Crime_Data.csv")
df_raw.head()

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Location 1,Total Incidents
0,11/12/2016,2:35:00,3B,300 SAINT PAUL PL,ROBBERY - STREET,O,NaN,111.0,CENTRAL,Downtown,"(39.2924100000, -76.6140800000)",1
1,11/12/2016,2:56:00,3CF,800 S BROADWAY,ROBBERY - COMMERCIAL,I,FIREARM,213.0,SOUTHEASTERN,Fells Point,"(39.2824200000, -76.5928800000)",1
2,11/12/2016,3:00:00,6D,1500 PENTWOOD RD,LARCENY FROM AUTO,O,NaN,413.0,NORTHEASTERN,Stonewood-Pentwood-Winston,"(39.3480500000, -76.5883400000)",1
3,11/12/2016,3:00:00,6D,6600 MILTON LN,LARCENY FROM AUTO,O,NaN,424.0,NORTHEASTERN,Westfield,"(39.3626300000, -76.5516100000)",1
4,11/12/2016,3:00:00,6E,300 W BALTIMORE ST,LARCENY,O,NaN,111.0,CENTRAL,Downtown,"(39.2893800000, -76.6197100000)",1


In [3]:
#change 'CrimeDate' format to date
from datetime import datetime
df_raw['CrimeDate'] = pd.to_datetime(df_raw['CrimeDate'])

In [ ]:
#create new variable for with month_year
df_raw["month_year"] = pd.to_datetime(df_raw['CrimeDate']).dt.to_period('M')
df_raw.head()
df_raw.to_csv('df_raw_baltimore.csv')

In [ ]:
#group count of incidents by CrimeDate
df_cc = df_raw.groupby(by=["month_year"]).count()
df_cc = df_cc.iloc[:, -1:]
df_cc = df_cc.rename(columns = {'Total Incidents':'Incidents'})
df_cc.to_csv('df_cc.csv')
df_cc= pd.read_csv('df_cc.csv')
print(df_cc)

In [ ]:
#remove 2016-11 for only having partial month
df_cc1 = df_cc.iloc[:-1]
print(df_cc1)

In [ ]:
#plot dataset
x = df_cc1['month_year']
y = df_cc1['Incidents']
plt.figure(figsize=(20,5))
plt.rcParams.update({'font.size':10})
plt.plot(x,y,)
plt.title('Baltimore Crime Incidents by Month', fontsize=20)
plt.xlabel('Date',fontsize=15)
plt.ylabel('Incidents',fontsize=15)
plt.xticks(rotation=90)
plt.figure(figsize=(5,5))
plt.rcParams.update({'font.size':10})
plt.show()

In [ ]:
#Augmented Dickey Fuller Test w/o differencing

from statsmodels.tsa.stattools import adfuller, acf, pacf

dftest = adfuller(df_cc1['Incidents'], autolag="AIC") #autolag = Method to use when automatically determining the lag

dfoutput = pd.Series(dftest[0:4], index=["Test Stats", "p-value", "# Lags", "# of obs"])
for key, value in dftest[4].items():
    dfoutput[f"Critical Value ({key})"] = value
print(dfoutput)
#Result cannot reject unit root

In [ ]:
# First-order difference
df_cc1['Incidents_diff'] = df_cc1['Incidents'].diff(periods=1)
df_cc2 = df_cc1.dropna()

# Plot
plt.figure(figsize=(15,5))
plt.rcParams.update({'font.size':10})
plt.title('Baltimore Crime Dataset with First-order difference', size=20)
plt.plot(df_cc2['Incidents'], label='Incidents')
plt.plot(df_cc2['Incidents_diff'], label='First-order difference', color='orange')
plt.legend();

In [ ]:
print(df_cc2) 

In [ ]:
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

#acf before first difference
plt.rcParams.update({'font.size':10})
plt.figure(figsize = (5,10))
plot_acf(df_cc2['Incidents'], lags=12)
plt.show()
#pacf after first difference
plot_pacf(df_cc2['Incidents'], lags=12)
plt.show()


In [ ]:
#acf after first difference
plt.rcParams.update({'font.size':10})
plt.figure(figsize = (5,10))
plot_acf(df_cc2['Incidents_diff'], lags=12)
plt.show()
#pacf after first difference
plot_pacf(df_cc2['Incidents_diff'], lags=12)
plt.show()


In [ ]:
#stepwise approach to find optimal ARIMA model
import pmdarima as pm

model = pm.auto_arima(df_cc2.Incidents, start_p=1, start_q=1,
                      test='adf',       # use adf test to find optimal 'd'
                      max_p=15, max_q=15, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
print(model.summary())
#result is ARIMA(1,0,0)

In [ ]:
#Augmented Dickey Fuller Test w/ differencing ARIMA(1,0,0)
from statsmodels.tsa.stattools import adfuller, acf, pacf

dftest = adfuller(df_cc2['Incidents_diff'], autolag="AIC") #autolag = Method to use when automatically determining the lag

dfoutput = pd.Series(dftest[0:4], index=["Test Stats", "p-value", "# Lags", "# of obs"])
for key, value in dftest[4].items():
    dfoutput[f"Critical Value ({key})"] = value
print(dfoutput)
#Result rejects for unit root

In [ ]:
### ARIMA(1,1,12) Model
from statsmodels.tsa.arima.model import ARIMA
from pandas import DataFrame

train = df_cc2.Incidents[:57] #first 57 observations
test  = df_cc2.Incidents[57:] #last 12 observations

p=1
d=1
q=12
model = ARIMA(train, order=(p, d, q))
fit_model = model.fit()
print(fit_model.summary())

# line plot of residuals
residuals = DataFrame(fit_model.resid)
residuals.plot()
plt.show()

# density plot of residuals
residuals.plot(kind='kde')
plt.show()

# summary stats of residuals
print(residuals.describe())

In [ ]:
#ploting forecast to actual with ARIMA (1,1,12) Model
fc = fit_model.forecast(13, alpha=0.05)  # 95% conf
fc_series = pd.Series(fc, index=test.index)

plt.figure(figsize=(12,5), dpi=100)
plt.plot(df_cc2.Incidents[:58], label='training')
plt.plot(test, label='actual')
plt.plot(fc_series, label='forecast')
plt.title('Forecast vs Actuals')
plt.legend(loc='lower left', fontsize=10)
plt.show()

In [ ]:
#ploting forecast to actual with ARIMA (1,0,0) Model
model_n1 = ARIMA(train, order=(1, 0, 0))
fit_model_n1 = model_n1.fit()

fc_n1 = fit_model_n1.forecast(12, alpha=0.05)  # 95% conf
fc_n1_series = pd.Series(fc_n1, index=test.index)

plt.figure(figsize=(12,5), dpi=100)
plt.plot(df_cc2.Incidents[:58], label='training')
plt.plot(test, label='actual')
plt.plot(fc_n1_series, label='forecast')
plt.title('Forecast vs Actuals')
plt.legend(loc='lower left', fontsize=10)
plt.show()

In [ ]:
##walk-forward analysis
from sklearn.metrics import mean_squared_error
from math import sqrt

# split into train and test sets
history = [x for x in train]
predictions = list()

## walk-forward validation
for t in range(58,70): 
    model1 = ARIMA(history, order=(p,d,q))
    model_fit = model1.fit()
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
    import warnings
    warnings.filterwarnings("ignore")
# evaluate forecasts
rmse = sqrt(mean_squared_error(test, predictions[-12:]))
print('Test RMSE: %.3f' % rmse)
# plot forecasts against actual outcomes
from matplotlib.ticker import MaxNLocator
plt.plot(list(test), label = 'Test')
plt.plot(predictions[-12:], color='red', label = 'Prediction')
plt.legend()
plt.show()